# Install Dependencies

In [1]:
!pip install pypdf langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/PrinciplesofFinance-WEB.pdf") #Loading the dataset
pages = loader.load()

In [5]:
len(pages)

643

In [6]:
page_text = ""

for i in range(0,len(pages)):
  text = pages[i].page_content
  page_text += text

In [7]:
print("Length of the characters:", len(page_text))

Length of the characters: 1628179


In [8]:
print(page_text[:2000])

	
	      Principles of Finance          SENIOR CONTRIBUTING AUTHORS JULIE DAHLQUIST, TEXAS CHRISTIAN UNIVERSITY RAINFORD KNIGHT, FLORIDA ATLANTIC UNIVERSITY          
OpenStax  
Rice University 
6100 Main Street MS- 375 
Houston, Texas 77005  
To learn more about OpenStax, visit http s://openstax.org.  
Individual print copies and bulk orders can be purchased through our website.  
©2022 Rice University.  Textbook content produced by OpenStax is licensed under a Creative Commons 
Attribution 4.0 International License  (CC BY 4.0) . Under this license, any user of  this textbook or the textbook 
contents herein must provide proper attribution as follows:   
-If you redistribute this textbook in a digital format (i ncluding but not limited to  PDF and HTML), then y ou
must retain on every page the following attribution:
“Access for free at openstax.org .”
-If you redistribute this textbook in a print format, then you must include on every physical page thefollowing attribution:
“Access f

In [9]:
chars = sorted(list(set(page_text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz|~©®µ×éêñó÷ŷ̃̄ρσ–—‘’“”•…™↓−≅⋯◦
121


In [10]:
# Characters to remove
characters_to_remove = "|~©®µ×éêñó÷ŷ̃̄ρσ–—‘’“”•…™↓−≅⋯◦"

# Remove the specified characters from the text
cleaned_text = ''.join([char for char in page_text if char not in characters_to_remove])

In [11]:
chars = sorted(list(set(cleaned_text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz
91


In [12]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("GPT Trial"))
print(decode(encode("GPT Trial")))

[41, 50, 54, 2, 54, 82, 73, 65, 76]
GPT Trial


In [14]:
import torch

data = torch.tensor(encode(cleaned_text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1624199]) torch.int64
tensor([ 0,  1,  0,  2,  2,  2,  2,  2,  2, 50, 82, 73, 78, 67, 73, 80, 76, 69,
        83,  2, 79, 70,  2, 40, 73, 78, 65, 78, 67, 69,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2, 53, 39, 48, 43, 49, 52,  2, 37, 49, 48, 54, 52, 43, 36,
        55, 54, 43, 48, 41,  2, 35, 55, 54, 42, 49, 52, 53,  2, 44, 55, 46, 43,
        39,  2, 38, 35, 42, 46, 51, 55, 43, 53, 54, 14,  2, 54, 39, 58, 35, 53,
         2, 37, 42, 52, 43, 53, 54, 43, 35, 48,  2, 55, 48, 43, 56, 39, 52, 53,
        43, 54, 59,  2, 52, 35, 43, 48, 40, 49, 52, 38,  2, 45, 48, 43, 41, 42,
        54, 14,  2, 40, 46, 49, 52, 43, 38, 35,  2, 35, 54, 46, 35, 48, 54, 43,
        37,  2, 55, 48, 43, 56, 39, 52, 53, 43, 54, 59,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  1, 49, 80, 69, 78, 53, 84, 65, 88,  2,  2,  1, 52, 73,
        67, 69,  2, 55, 78, 73, 86, 69, 82, 83, 73, 84, 89,  2,  1, 24, 19, 18,
        18,  2, 47, 65, 73, 78,  2, 53, 84, 82, 69, 69, 84,  2, 47, 53, 15,  2,
      

In [15]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [16]:
block_size = 12
train_data[:block_size+1]

tensor([ 0,  1,  0,  2,  2,  2,  2,  2,  2, 50, 82, 73, 78])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"When input is {context} the target: {target}")

When input is tensor([0]) the target: 1
When input is tensor([0, 1]) the target: 0
When input is tensor([0, 1, 0]) the target: 2
When input is tensor([0, 1, 0, 2]) the target: 2
When input is tensor([0, 1, 0, 2, 2]) the target: 2
When input is tensor([0, 1, 0, 2, 2, 2]) the target: 2
When input is tensor([0, 1, 0, 2, 2, 2, 2]) the target: 2
When input is tensor([0, 1, 0, 2, 2, 2, 2, 2]) the target: 2
When input is tensor([0, 1, 0, 2, 2, 2, 2, 2, 2]) the target: 50
When input is tensor([ 0,  1,  0,  2,  2,  2,  2,  2,  2, 50]) the target: 82
When input is tensor([ 0,  1,  0,  2,  2,  2,  2,  2,  2, 50, 82]) the target: 73
When input is tensor([ 0,  1,  0,  2,  2,  2,  2,  2,  2, 50, 82, 73]) the target: 78
